In [12]:
import gif
import json
import pandas as pd
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
# from plotly_gif import GIF, capture
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [13]:
with open('./data/test/evjvqa_public_test-lang-qtype-answer.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_annotations = test_data['annotations']

In [59]:
en_answer_lengths = []
vi_answer_lengths = []
ja_answer_lengths = []

for anno in tqdm(test_annotations):
    if anno['question_type'] == 'OTHERS':
        if anno['language'] in ['en']:
            awords = anno['answer'].split()
            anno['answer_length'] = len(awords)

            if len(awords) <= 5:
                en_answer_lengths.append(f"{len(awords)} words")
            elif 5 < len(awords) <= 10:
                en_answer_lengths.append('> 5 words')
            elif len(awords) > 10:
                en_answer_lengths.append('> 10 words')

    #         en_answer_lengths.append(len(awords))
        elif anno['language'] in ['vi']:
            awords = anno['answer'].split()
            anno['answer_length'] = len(awords)

            if len(awords) <= 5:
                vi_answer_lengths.append(f"{len(awords)} words")
            elif 5 < len(awords) <= 10:
                vi_answer_lengths.append('> 5 words')
            elif len(awords) > 10:
                vi_answer_lengths.append('> 10 words')

    #         vi_answer_lengths.append(len(awords))
        elif anno['language'] in ['ja']:
            anno['answer_length'] = len(anno['answer'])

            if len(anno['answer']) <= 5:
                ja_answer_lengths.append(f"{len(anno['answer'])} words")
            elif 5 < len(anno['answer']) <= 10:
                ja_answer_lengths.append('> 5 words')
            elif len(anno['answer']) > 10:
                ja_answer_lengths.append('> 10 words')

    #         ja_answer_lengths.append(len(anno['answer']))

  0%|          | 0/5015 [00:00<?, ?it/s]

In [60]:
df_en = pd.DataFrame.from_dict({'answer_length': en_answer_lengths})

df_vi = pd.DataFrame.from_dict({'answer_length': vi_answer_lengths})

df_ja = pd.DataFrame.from_dict({'answer_length': ja_answer_lengths})

def visualize(df, name_file):
    #labels
    lab = df["answer_length"].value_counts().keys().tolist()
    #values
    val = df["answer_length"].value_counts().values.tolist()
    trace = go.Pie(labels=lab, 
                    values=val, 
                    marker=dict(colors=['red']), 
                    # Seting values to 
                    hoverinfo="value"
                  )
    data = [trace]

    #set title 
    layout = go.Layout(title="Answer Length Distribution")

    fig = go.Figure(data = data,layout = layout)

    iplot(fig)
    
    fig.write_image(name_file)

In [61]:
visualize(df_en, 'fig-answer-length-english.png')

In [62]:
visualize(df_vi, 'fig-answer-length-vietnamese.png')

In [63]:
visualize(df_ja, 'fig-answer-length-japanese.png')